In [ ]:
def P_picking(st_main, adj_st_1, adj_st_2, ventana_10s, ventana_30s, nsta, nlta, v_P, coords_st, thr_on, thr_off):
    """
    Función que realiza el picking de la onda P. En caso de que se detecte un sismo en las estación principal,
    se verifica que las estaciones adyacentes también lo detecten.

    Entradas:
    ----------
    st_main: obspy.core.stream.Stream
        Stream de la estación principal.
    adj_st_1: obspy.core.stream.Stream
        Stream de la estación adyacente 1.
    adj_st_2: obspy.core.stream.Stream
        Stream de la estación adyacente 2.
    ventana_10s: int
        Tiempo de la ventana en segundos donde se aplica el STA/LTA.
    ventana_30s: int
        Tiempo de la señal en segundos que se analiza en cada iteración.
    nsta: int
        Largo del short time average en segundos.
    nlta: int
        Largo del long time average en segundos.
    v_P: float
        Velocidad de propagación de la onda P en km/s.
    coords_st: list
        Coordenadas (latitud, longitud) de las estaciones.
    
    Salidas:
    ----------
    time_trigger_main: list
        Lista que contiene tiempos cuando solo la estación principal detecta un evento
    time_trigger_all: list
        Lista que contiene tiempos cuando todas las estaciones detectan un evento
    """


    # Coordenadas (latitud, longitud) de las estaciones
    coord_main = coords_st[0]
    coord_adj_1 = coords_st[1]
    coord_adj_2 = coords_st[2]

    # Distancia entre las estaciones
    d_main_adj1 = geodesic(coord_main, coord_adj_1).kilometers
    d_main_adj2 = geodesic(coord_main, coord_adj_2 ).kilometers
    d_adj1_adj2 = geodesic(coord_adj_1 , coord_adj_2 ).kilometers

    # Tiempo de llegada de la onda P a las estaciones
    t_P_main_adj1 = d_main_adj1 / v_P
    t_P_main_adj2 = d_main_adj2 / v_P
    t_P_adj1_adj2 = d_adj1_adj2 / v_P

    # Elección de canal para cada estación. 
    tr_main = st_main.select(channel='HHZ')[0]
    tr_adj_1 = adj_st_1.select(channel='HHZ')[0]
    tr_adj_2 = adj_st_2.select(channel='HHZ')[0]

    # Frecuencia de muestreo
    fs = tr_main.stats.sampling_rate

    # Convertir los tamaños de ventana a muestras
    muestras_10s = int(ventana_10s * fs)
    muestras_30s = int(ventana_30s * fs)

    # Inicializar el cálculo STA/LTA para los primeros 30 segundos de la traza.
    cft_main = inicializar_sta_lta(tr_main, int(nsta * fs), int(nlta * fs))
    initial_tr_main = tr_main.slice(endtime=tr_main.stats.starttime + ventana_30s)

    time_trigger_main = []
    time_trigger_all = []

    
    for i in range(10, len(tr_main), muestras_10s):
        # Tomar la sección actual de 30 segundos
        end_window = i + muestras_30s
        if end_window > len(tr_main):
            #print(end_window)
            break
        new_tr_main = tr_main.slice(starttime = tr_main.stats.starttime + i/fs, endtime = tr_main.stats.starttime + end_window/fs)
        cft_main = actualizar_sta_lta(cft_main, initial_tr_main , new_tr_main, int(nsta * fs), int(nlta * fs))

        # Si se activa se plotea
        if np.any(cft_main > thr_on):
            # No aseguramos de que el tiempo de inicio del sismo no se haya registrado antes
            if new_tr_main.stats.starttime not in time_trigger_main:
                time_trigger_main.append(new_tr_main.stats.starttime)
            
            #plot_trigger(nueva_traza,cft_main, thr_on, thr_off)
            #time_trigger_main.append(new_tr_main.stats.starttime)

            # Verificar si las estaciones adyacentes también detectan un sismo en ese intervalo de tiempo
            initial_tr_adj_1 = tr_adj_1.slice(starttime = new_tr_main.stats.starttime, endtime = new_tr_main.stats.endtime)
            initial_tr_adj_2 = tr_adj_2.slice(starttime = new_tr_main.stats.starttime, endtime = new_tr_main.stats.endtime)
            cft_adj_1 = inicializar_sta_lta(initial_tr_adj_1, int(nsta * fs), int(nlta * fs))
            cft_adj_2 = inicializar_sta_lta(initial_tr_adj_2, int(nsta * fs), int(nlta * fs))
            
            for j in range(math.ceil(max(t_P_main_adj1, t_P_main_adj2)/10)+1):
                #print(j*muestras_10s, (j+1)*muestras_30s)
                new_tr_adj_1 = tr_adj_1.slice(starttime = new_tr_main.stats.starttime + j*ventana_10s, endtime = new_tr_main.stats.endtime + j*ventana_10s)
                new_tr_adj_2 = tr_adj_2.slice(starttime = initial_tr_adj_2.stats.starttime + j*ventana_10s, endtime = new_tr_main.stats.endtime + j*ventana_10s)
                cfr_adj_1 = actualizar_sta_lta(cft_adj_1, initial_tr_adj_1, new_tr_adj_1, int(nsta * fs), int(nlta * fs))
                cfr_adj_2 = actualizar_sta_lta(cft_adj_2, initial_tr_adj_2, new_tr_adj_2, int(nsta * fs), int(nlta * fs))

                if np.any(cft_adj_1 > thr_on) and np.any(cft_adj_2 > thr_on):
                    #plot_trigger(new_tr_main,cft_main, thr_on, thr_off)
                    #initial_tr_adj_1 = new_tr_adj_1
                    #initial_tr_adj_2 = new_tr_adj_2
                    if new_tr_main.stats.starttime not in time_trigger_all:
                        time_trigger_all.append(new_tr_main.stats.starttime)
    

            initial_tr_adj_1 = new_tr_adj_1
            initial_tr_adj_2 = new_tr_adj_2


        initial_tr_main  = new_tr_main
        
    return time_trigger_main, time_trigger_all
    
    
    

In [ ]:
# Junta los archivos por carpeta. Queda un diccionario que tiene como llave la carpeta, 
# y como valor una lista con los paths a los archivos de esa carpeta.
def sort_files(file_dic, extension):
    # Diccionario para almacenar los archivos por carpeta
    grouped_files = defaultdict(list)
    key_names = []
    for file in file_dic[extension]:
        # Extraer la carpeta del path del archivo
        parts = file.split("\\")
        folder = parts[-2]  # La carpeta es el penúltimo elemento en el path
        if [folder] not in key_names:
            key_names.append([folder])
        # Agrupar los archivos
        grouped_files[folder].append(file)

    key_names = list(itertools.chain(*key_names))
    return grouped_files, key_names

In [ ]:
# Coordenadas (latitud, longitud) de las estaciones
coord_C010 = (-29.24, -71.46)
coord_AC04 = (-28.20, -71.07)
coord_AC05 = (-28.84, -70.27)


# Distancia entre las estaciones
d_AC04_AC05 = geodesic(coord_AC04, coord_AC05).kilometers
d_AC04_C010 = geodesic(coord_AC04, coord_C010).kilometers
d_AC05_C010 = geodesic(coord_AC05, coord_C010).kilometers

# Velocidad de propagación de las ondas P en kilometros por segundo
v_P = 8.046

# Tiempo de llegada de la onda P a las estaciones
t_P_AC04_AC05 = d_AC04_AC05 / v_P
t_P_AC05_C010 = d_AC05_C010 / v_P
t_P_AC04_C010 = d_AC04_C010 / v_P


#### Esto se usa si se quiere usar los archivos de 24hrs

In [ ]:
#estación principal
main_st = files_24hrs_ch[key_names_24hrs[2]]
st_main = read(main_st[0])
st_main += read(main_st[1])
st_main += read(main_st[2])
st_main.filter('bandpass', freqmin=4.0, freqmax=10.0, corners=1, zerophase=True) 
print(st_main.select(channel='HHZ'))

#estación adyacente
adj_st_1 = files_24hrs_ch[key_names_24hrs[0]]
st_adj_1 = read(adj_st_1[0])
st_adj_1 += read(adj_st_1[1])
st_adj_1 += read(adj_st_1[2])
st_adj_1.filter('bandpass', freqmin=4.0, freqmax=10.0, corners=1, zerophase=True)
print(st_adj_1.select(channel='HHZ'))

#estación adyacente
adj_st_2 = files_24hrs_ch[key_names_24hrs[1]]
st_adj_2 = read(adj_st_2[0])
st_adj_2 += read(adj_st_2[1])
st_adj_2 += read(adj_st_2[2])
st_adj_2.filter('bandpass', freqmin=4.0, freqmax=10.0, corners=1, zerophase=True)
print(st_adj_2.select(channel='HHZ'))


### Esto es para una estación principal y 2 adyacentes, ahora estoy viendo para un número variable de estaciones adyacentes

In [ ]:
def P_picking(st_main, adj_st_1, adj_st_2, ventana_10s, ventana_30s, nsta, nlta, v_P, coords_st, thr_on, thr_off):
    """
    Función que realiza el picking de la onda P. En caso de que se detecte un sismo en las estación principal,
    se verifica que las estaciones adyacentes también lo detecten.

    Entradas:
    ----------
    st_main: obspy.core.stream.Stream
        Stream de la estación principal.
    adj_st_1: obspy.core.stream.Stream
        Stream de la estación adyacente 1.
    adj_st_2: obspy.core.stream.Stream
        Stream de la estación adyacente 2.
    ventana_10s: int
        Tiempo de la ventana en segundos donde se aplica el STA/LTA.
    ventana_30s: int
        Tiempo de la señal en segundos que se analiza en cada iteración.
    nsta: int
        Largo del short time average en segundos.
    nlta: int
        Largo del long time average en segundos.
    v_P: float
        Velocidad de propagación de la onda P en km/s.
    coords_st: list
        Coordenadas (latitud, longitud) de las estaciones.
    
    Salidas:
    ----------
    time_trigger_main: list
        Lista que contiene tiempos cuando solo la estación principal detecta un evento
    time_trigger_all: list
        Lista que contiene tiempos cuando todas las estaciones detectan un evento
    """


    # Coordenadas (latitud, longitud) de las estaciones
    coord_main = coords_st[0]
    coord_adj_1 = coords_st[1]
    coord_adj_2 = coords_st[2]

    # Distancia entre las estaciones
    d_main_adj1 = geodesic(coord_main, coord_adj_1).kilometers
    d_main_adj2 = geodesic(coord_main, coord_adj_2 ).kilometers
    d_adj1_adj2 = geodesic(coord_adj_1 , coord_adj_2 ).kilometers

    # Tiempo de llegada de la onda P a las estaciones
    t_P_main_adj1 = d_main_adj1 / v_P
    t_P_main_adj2 = d_main_adj2 / v_P
    t_P_adj1_adj2 = d_adj1_adj2 / v_P

    # Elección de canal para cada estación. 
    tr_main = st_main.select(channel='BHZ')[0]
    tr_adj_1 = adj_st_1.select(channel='BHZ')[0]
    tr_adj_2 = adj_st_2.select(channel='BHZ')[0]

    # Frecuencia de muestreo
    fs = tr_main.stats.sampling_rate

    # Convertir los tamaños de ventana a muestras
    muestras_10s = int(ventana_10s * fs)
    muestras_30s = int(ventana_30s * fs)

    # Inicializar el cálculo STA/LTA para los primeros 30 segundos de la traza.
    cft_main = inicializar_sta_lta(tr_main, int(nsta * fs), int(nlta * fs))
    initial_tr_main = tr_main.slice(endtime=tr_main.stats.starttime + ventana_30s)

    time_trigger_main = []
    time_trigger_all = []

    for i in range(10, len(tr_main), muestras_10s):
        # Tomar la sección actual de 30 segundos
        end_window = i + muestras_30s
        if end_window > len(tr_main):
            break
        new_tr_main = tr_main.slice(starttime = tr_main.stats.starttime + i/fs, endtime = tr_main.stats.starttime + end_window/fs)
        cft_main = actualizar_sta_lta(cft_main, initial_tr_main , new_tr_main, int(nsta * fs), int(nlta * fs))

        # Si se activa se plotea
        if np.any(cft_main > thr_on):
            # Convertimos starttime a una cadena que solo contiene la fecha, la hora y los minutos
            time_main_str = new_tr_main.stats.starttime.strftime("%Y-%m-%dT%H:%M")
            
            # Creamos una lista separada para las comparaciones
            time_trigger_main_comp = [t.strftime("%Y-%m-%dT%H:%M") for t in time_trigger_main]
            
            # No aseguramos de que el tiempo de inicio del sismo no se haya registrado antes
            if time_main_str not in time_trigger_main_comp:
                time_trigger_main.append(new_tr_main.stats.starttime)
            
            #plot_trigger(nueva_traza,cft_main, thr_on, thr_off)
            #time_trigger_main.append(new_tr_main.stats.starttime)

            # Verificar si las estaciones adyacentes también detectan un sismo en ese intervalo de tiempo
            initial_tr_adj_1 = tr_adj_1.slice(starttime = new_tr_main.stats.starttime, endtime = new_tr_main.stats.endtime)
            initial_tr_adj_2 = tr_adj_2.slice(starttime = new_tr_main.stats.starttime, endtime = new_tr_main.stats.endtime)
            cft_adj_1 = inicializar_sta_lta(initial_tr_adj_1, int(nsta * fs), int(nlta * fs))
            cft_adj_2 = inicializar_sta_lta(initial_tr_adj_2, int(nsta * fs), int(nlta * fs))
            
            for j in range(math.ceil(max(t_P_main_adj1, t_P_main_adj2)/10)+1):
                #print(j*muestras_10s, (j+1)*muestras_30s)
                new_tr_adj_1 = tr_adj_1.slice(starttime = new_tr_main.stats.starttime + j*ventana_10s, endtime = new_tr_main.stats.endtime + j*ventana_10s)
                new_tr_adj_2 = tr_adj_2.slice(starttime = new_tr_main.stats.starttime + j*ventana_10s, endtime = new_tr_main.stats.endtime + j*ventana_10s)
                cft_adj_1 = actualizar_sta_lta(cft_adj_1, initial_tr_adj_1, new_tr_adj_1, int(nsta * fs), int(nlta * fs))
                cft_adj_2 = actualizar_sta_lta(cft_adj_2, initial_tr_adj_2, new_tr_adj_2, int(nsta * fs), int(nlta * fs))

                if np.any(cft_adj_1 > thr_on) and np.any(cft_adj_2 > thr_on):
                    #plot_trigger(new_tr_main,cft_main, thr_on, thr_off)
                    #initial_tr_adj_1 = new_tr_adj_1
                    #initial_tr_adj_2 = new_tr_adj_2
                    time_all_str = new_tr_main.stats.starttime.strftime("%Y-%m-%dT%H:%M")
                    time_trigger_all_comp = [t.strftime("%Y-%m-%dT%H:%M") for t in time_trigger_all]
                    if time_all_str not in time_trigger_all_comp:
                        time_trigger_all.append(new_tr_main.stats.starttime)
    

            initial_tr_adj_1 = new_tr_adj_1
            initial_tr_adj_2 = new_tr_adj_2


        initial_tr_main  = new_tr_main
        
    return time_trigger_main, time_trigger_all
    

In [ ]:
#Código para obtener en time_events_xlsx.txt solo los tiempos de evntos_catalogo.xlsx
df = pd.read_excel('24hrs basura/evntos _catalogo.xlsx')
times = df['Fecha UTC'].tolist()
# Convierte la lista al formato UTC
formatted_times = [time.strftime('%Y-%m-%dT%H:%M:%S') for time in times if pd.notnull(time)]
with open('times_events_xlsx.txt', 'w') as archivo:
    archivo.write('\n'.join(formatted_times))

#Código para obtener en time_events.txt solo los tiempos de query.txt
with open(f'24hrs basura\query.txt', 'r') as archivo:
    lines = archivo.readlines()
times = [line.split('|')[1] for line in lines]
with open('times_events.txt', 'w') as archivo:
    archivo.write('\n'.join(times)) 


In [ ]:

# rango de parámetros a probar
nsta_values = [1.5, 2, 2.5, 3]
nlta_values = [10, 12, 15, 20]
thr_on_values = [3.5, 4, 5, 5.8]
thr_off = 3

# inicialización de los mejores parámetros y el mejor resultado
best_params = (nsta_values[0], nlta_values[0], thr_on_values[0])
best_result = -float('inf')  # El resultado más alto es el mejor 

# Calcular el número total de iteraciones
total = len(nsta_values) * len(nlta_values) * len(thr_on_values)

# Crear un objeto tqdm
pbar = tqdm(total=total)

# se itera sobre los parámetros
for nsta in nsta_values:
    for nlta in nlta_values:
        for thr_on in thr_on_values:
            # Llamar a la función P_picking con los parámetros actuales
            time_main, time_all = P_picking(stations, ventana_10s, ventana_30s, nsta, nlta, v_P, coord_list, thr_on, thr_off)
            
            # Guardar los tiempos predichos
            with open('time_trigger.txt', 'w') as archivo:
                archivo.write('\n'.join(str(time) for time in time_all))
            
            # Leer los tiempos reales y predichos
            with open('times_events_xlsx.txt', 'r') as f:
                tiempos_reales = [datetime.strptime(line.strip(), '%Y-%m-%dT%H:%M:%S') for line in f]
            with open('time_trigger.txt', 'r') as f:
                tiempos_predichos = [datetime.strptime(line.strip(), '%Y-%m-%dT%H:%M:%S.%fZ') for line in f]
            
            # Calcular los conjuntos de tiempos reales y predichos
            conjunto_reales = set([t.replace(second=0) for t in tiempos_reales])
            conjunto_predichos = set([t.replace(second=0, microsecond=0) for t in tiempos_predichos])
            
            # Calcular los verdaderos positivos, falsos positivos y falsos negativos
            verdaderos_positivos = conjunto_reales & conjunto_predichos
            falsos_positivos = conjunto_predichos - conjunto_reales
            falsos_negativos = conjunto_reales - conjunto_predichos
            
            # Calcular el resultado (por ejemplo, el número de falsos positivos). Usar recall como métrica
            result = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_negativos))  # Aquí puedes cambiar la métrica de acuerdo a tus necesidades
            
            # Si el resultado actual es mejor que el mejor resultado hasta ahora, actualizar los mejores parámetros y el mejor resultado
            if result > best_result:
                best_params = (nsta, nlta, thr_on, thr_off)
                best_result = result
            # Actualizar la barra de progreso
            pbar.update()

pbar.close()


# FUNCION OPTIMIZE PARAMETERS QUE SE QUE FUNCIONA

In [ ]:
def optimize_parameters(nsta_values, nlta_values, thr_on_values, thr_off, stations, ventana_10s, ventana_30s, v_P, coord_list):
    # Diccionario para almacenar los mejores parámetros y el resultado de f1-score
    best_params = {}

    # Número total de iteraciones
    total = len(nsta_values) * len(nlta_values) * len(thr_on_values)

    pbar = tqdm(total=total, leave=True)

    with open('times_events_24hrs.txt', 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Saltar la cabecera
        tiempos_reales = [datetime.strptime(row[1], '%Y-%m-%dT%H:%M:%S') for row in reader]

    # se itera sobre los parámetros
    for nsta, nlta, thr_on in product(nsta_values, nlta_values, thr_on_values):
        # Llamar a la función P_picking con los parámetros actuales
        time_main, time_all = P_picking(stations, ventana_10s, ventana_30s, nsta, nlta, v_P, coord_list, thr_on, thr_off)

        # Guardar los tiempos predichos
        tiempos_predichos = [datetime.strptime(str(time), '%Y-%m-%dT%H:%M:%S.%fZ') for time in time_all]
        
        # Calcular los conjuntos de tiempos reales y predichos
        conjunto_reales = set([t.replace(second=0) for t in tiempos_reales])
        conjunto_predichos = set([t.replace(second=0, microsecond=0) for t in tiempos_predichos])
        
        # Calcular los verdaderos positivos, falsos positivos y falsos negativos
        verdaderos_positivos = conjunto_reales & conjunto_predichos
        falsos_positivos = conjunto_predichos - conjunto_reales
        falsos_negativos = conjunto_reales - conjunto_predichos
        
        # Usar F1-score como métrica de evaluación para minimizar los falsos positivos y falsos negativos
        if verdaderos_positivos:
            presicion = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_positivos))
            recall = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_negativos))
            f1_score = 2 * (presicion * recall) / (presicion + recall)
        else:
            f1_score = 0  # Si no hay verdaderos positivos ni falsos positivos, la precisión es 0

        # Si el resultado actual es mejor que el mejor resultado hasta ahora, actualizar los mejores parámetros y el mejor resultado
        if len(best_params) < 5 or f1_score > min(best_params.values()):
            if len(best_params) == 5:
                # eliminar la combinación de parámetros con el peor resultado
                worst_key = min(best_params, key=best_params.get)
                del best_params[worst_key]   
            # agregar la nueva combinación de parámetros y su resultado al diccionario
            best_params[(nsta, nlta, thr_on, thr_off)] = f1_score 
            ic(best_params)

        # Actualizar la barra de progreso
        pbar.update()

    pbar.close()

    return best_params

### Este es un intento de usar multiprocessing

In [ ]:
import concurrent.futures

# rango de parámetros a probar
nsta_values = [1.5, 1.8, 2, 2.3 ,2.5, 2.8, 3, 3.5]
nlta_values = [8, 10, 12, 15, 17, 20, 23, 25]
thr_on_values = [3.5, 3.9, 4, 4.5, 5, 5.3, 5.5, 5.8]
thr_off = 3 # solo sirve al momento de hacer plot_trigger, por lo que no es necesario cambiarlo para buscar eventos

# Diccionario para almacenar los mejores parámetros y el resultado de f1-score
best_params = {}

# Número total de iteraciones
total = len(nsta_values) * len(nlta_values) * len(thr_on_values)

pbar = tqdm(total=total, leave=True)

with open('times_events_24hrs.txt', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Saltar la cabecera
    tiempos_reales = [datetime.strptime(row[1], '%Y-%m-%dT%H:%M:%S') for row in reader]

# Función para calcular el f1-score para una combinación de parámetros
def calculate_f1_score(params):
    nsta, nlta, thr_on = params
    # Llamar a la función P_picking con los parámetros actuales
    time_main, time_all = P_picking(stations, ventana_10s, ventana_30s, nsta, nlta, v_P, coord_list, thr_on, thr_off)

    # Guardar los tiempos predichos
    tiempos_predichos = [datetime.strptime(str(time), '%Y-%m-%dT%H:%M:%S.%fZ') for time in time_all]
    
    # Calcular los conjuntos de tiempos reales y predichos
    conjunto_reales = set([t.replace(second=0) for t in tiempos_reales])
    conjunto_predichos = set([t.replace(second=0, microsecond=0) for t in tiempos_predichos])
    
    # Calcular los verdaderos positivos, falsos positivos y falsos negativos
    verdaderos_positivos = conjunto_reales & conjunto_predichos
    falsos_positivos = conjunto_predichos - conjunto_reales
    falsos_negativos = conjunto_reales - conjunto_predichos
    
    # Usar F1-score como métrica de evaluación para minimizar los falsos positivos y falsos negativos
    if len(verdaderos_positivos) > 0:
        presicion = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_positivos))
        recall = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_negativos))
        f1_score = 2 * (presicion * recall) / (presicion + recall)
    else:
        f1_score = 0  # Si no hay verdaderos positivos ni falsos positivos, la precisión es 0

    return (nsta, nlta, thr_on, thr_off), f1_score

# Crear una lista de todas las combinaciones de parámetros
params = [(nsta, nlta, thr_on) for nsta in nsta_values for nlta in nlta_values for thr_on in thr_on_values]

# Crear un pool de trabajadores y calcular el f1-score para todas las combinaciones de parámetros
with concurrent.futures.ProcessPoolExecutor() as executor:
    for params, f1_score in executor.map(calculate_f1_score, params):
        # Si el resultado actual es mejor que el mejor resultado hasta ahora, actualizar los mejores parámetros y el mejor resultado
        if len(best_params) < 5 or f1_score > min(best_params.values()):
            if len(best_params) == 5:
                # eliminar la combinación de parámetros con el peor resultado
                worst_key = min(best_params, key=best_params.get)
                del best_params[worst_key]   
            # agregar la nueva combinación de parámetros y su resultado al diccionario
            best_params[params] = f1_score 
            ic(best_params)

        # Actualizar la barra de progreso
        pbar.update()

pbar.close()


### Lo siguiente es el código que funciona para buscar los parámetros usando fuerza bruta. Es la última versión antes de ver como hacerlo más óptimo partiendo por paralelización

In [ ]:

# rango de parámetros a probar
nsta_values = [1.5, 1.8, 2, 2.3 ,2.5, 2.8, 3, 3.5]
nlta_values = [8, 10, 12, 15, 17, 20, 23, 25]
thr_on_values = [3.5, 3.9, 4, 4.5, 5, 5.3, 5.5, 5.8]
thr_off = 3 # solo sirve al momento de hacer plot_trigger, por lo que no es necesario cambiarlo para buscar eventos

# Diccionario para almacenar los mejores parámetros y el resultado de f1-score
best_params = {}

# Número total de iteraciones
total = len(nsta_values) * len(nlta_values) * len(thr_on_values)

pbar = tqdm(total=total, leave=True)

with open('times_events_24hrs.txt', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Saltar la cabecera
    tiempos_reales = [datetime.strptime(row[1], '%Y-%m-%dT%H:%M:%S') for row in reader]

# se itera sobre los parámetros
for nsta, nlta, thr_on in product(nsta_values, nlta_values, thr_on_values):
    # Llamar a la función P_picking con los parámetros actuales
    time_main, time_all = P_picking(stations, ventana_10s, ventana_30s, nsta, nlta, v_P, coord_list, thr_on, thr_off)

    # Guardar los tiempos predichos
    tiempos_predichos = [datetime.strptime(str(time), '%Y-%m-%dT%H:%M:%S.%fZ') for time in time_all]
    
    # Calcular los conjuntos de tiempos reales y predichos
    conjunto_reales = set([t.replace(second=0) for t in tiempos_reales])
    conjunto_predichos = set([t.replace(second=0, microsecond=0) for t in tiempos_predichos])
    
    # Calcular los verdaderos positivos, falsos positivos y falsos negativos
    verdaderos_positivos = conjunto_reales & conjunto_predichos
    falsos_positivos = conjunto_predichos - conjunto_reales
    falsos_negativos = conjunto_reales - conjunto_predichos
    
    # Usar F1-score como métrica de evaluación para minimizar los falsos positivos y falsos negativos
    if verdaderos_positivos:
        presicion = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_positivos))
        recall = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_negativos))
        f1_score = 2 * (presicion * recall) / (presicion + recall)
    else:
        f1_score = 0  # Si no hay verdaderos positivos ni falsos positivos, la precisión es 0

    # Si el resultado actual es mejor que el mejor resultado hasta ahora, actualizar los mejores parámetros y el mejor resultado
    if len(best_params) < 5 or f1_score > min(best_params.values()):
        if len(best_params) == 5:
            # eliminar la combinación de parámetros con el peor resultado
            worst_key = min(best_params, key=best_params.get)
            del best_params[worst_key]   
        # agregar la nueva combinación de parámetros y su resultado al diccionario
        best_params[(nsta, nlta, thr_on, thr_off)] = f1_score 
        ic(best_params)

    # Actualizar la barra de progreso
    pbar.update()

pbar.close()


### OTRA FORMA DE HACER EL MÓDULO optimize_parameters.py usando multiprocessing


In [ ]:

def worker(nsta, nlta, thr_on, thr_off, stations, ventana_10s, ventana_30s, v_P, coord_list, tiempos_reales):
    # Llamar a la función P_picking con los parámetros actuales
    time_main, time_all = P_picking(stations, ventana_10s, ventana_30s, nsta, nlta, v_P, coord_list, thr_on, thr_off)

    # Guardar los tiempos predichos
    tiempos_predichos = [datetime.strptime(str(time), '%Y-%m-%dT%H:%M:%S.%fZ') for time in time_all]
    
    # Calcular los conjuntos de tiempos reales y predichos
    conjunto_reales = set([t.replace(second=0) for t in tiempos_reales])
    conjunto_predichos = set([t.replace(second=0, microsecond=0) for t in tiempos_predichos])
    
    # Calcular los verdaderos positivos, falsos positivos y falsos negativos
    verdaderos_positivos = conjunto_reales & conjunto_predichos
    falsos_positivos = conjunto_predichos - conjunto_reales
    falsos_negativos = conjunto_reales - conjunto_predichos
    
    # Usar F1-score como métrica de evaluación para minimizar los falsos positivos y falsos negativos
    if verdaderos_positivos:
        presicion = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_positivos))
        recall = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_negativos))
        f1_score = 2 * (presicion * recall) / (presicion + recall)
    else:
        f1_score = 0  # Si no hay verdaderos positivos ni falsos positivos, la precisión es 0

    return (nsta, nlta, thr_on, thr_off), f1_score

def optimize_parameters(nsta_values, nlta_values, thr_on_values, thr_off, stations, ventana_10s, ventana_30s, v_P, coord_list):
    # Diccionario para almacenar los mejores parámetros y el resultado de f1-score
    best_params = {}

    # Número total de iteraciones
    total = len(nsta_values) * len(nlta_values) * len(thr_on_values)

    pbar = tqdm(total=total, leave=True)

    with open('times_events_24hrs.txt', 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Saltar la cabecera
        tiempos_reales = [datetime.strptime(row[1], '%Y-%m-%dT%H:%M:%S') for row in reader]

    # se itera sobre los parámetros
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(worker, nsta, nlta, thr_on, thr_off, stations, ventana_10s, ventana_30s, v_P, coord_list, tiempos_reales): (nsta, nlta, thr_on, thr_off) for nsta, nlta, thr_on in product(nsta_values, nlta_values, thr_on_values)}
        for future in concurrent.futures.as_completed(futures):
            params = futures[future]
            try:
                result = future.result()
            except Exception as exc:
                print('%r generated an exception: %s' % (params, exc))
            else:
                if len(best_params) < 5 or result[1] > min(best_params.values()):
                    if len(best_params) == 5:
                        # eliminar la combinación de parámetros con el peor resultado
                        worst_key = min(best_params, key=best_params.get)
                        del best_params[worst_key]   
                    # agregar la nueva combinación de parámetros y su resultado al diccionario
                    best_params[result[0]] = result[1] 

            # Actualizar la barra de progreso
            pbar.update()

    pbar.close()

    return best_params


### La siguiente es la función optimize_parameters que se que funciona


In [ ]:

def optimize_parameters(nsta_values, nlta_values, thr_on_values, thr_off, stations, ventana_10s, ventana_30s, v_P, coord_list):
    # Diccionario para almacenar los mejores parámetros y el resultado de f1-score
    best_params = {}

    # Número total de iteraciones
    total = len(nsta_values) * len(nlta_values) * len(thr_on_values)

    pbar = tqdm(total=total, leave=True)

    with open('times_events_24hrs.txt', 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Saltar la cabecera
        tiempos_reales = [datetime.strptime(row[1], '%Y-%m-%dT%H:%M:%S') for row in reader]

    # se itera sobre los parámetros
    for nsta, nlta, thr_on in product(nsta_values, nlta_values, thr_on_values):
        # Llamar a la función P_picking con los parámetros actuales
        time_main, time_all = P_picking(stations, ventana_10s, ventana_30s, nsta, nlta, v_P, coord_list, thr_on, thr_off)

        # Guardar los tiempos predichos
        tiempos_predichos = [datetime.strptime(str(time), '%Y-%m-%dT%H:%M:%S.%fZ') for time in time_all]
        
        # Calcular los conjuntos de tiempos reales y predichos
        conjunto_reales = set([t.replace(second=0) for t in tiempos_reales])
        conjunto_predichos = set([t.replace(second=0, microsecond=0) for t in tiempos_predichos])
        
        # Calcular los verdaderos positivos, falsos positivos y falsos negativos
        verdaderos_positivos = conjunto_reales & conjunto_predichos
        falsos_positivos = conjunto_predichos - conjunto_reales
        falsos_negativos = conjunto_reales - conjunto_predichos
        
        # Usar F1-score como métrica de evaluación para minimizar los falsos positivos y falsos negativos
        if verdaderos_positivos:
            presicion = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_positivos))
            recall = len(verdaderos_positivos) / (len(verdaderos_positivos) + len(falsos_negativos))
            f1_score = 2 * (presicion * recall) / (presicion + recall)
        else:
            f1_score = 0  # Si no hay verdaderos positivos ni falsos positivos, la precisión es 0

        # Si el resultado actual es mejor que el mejor resultado hasta ahora, actualizar los mejores parámetros y el mejor resultado
        if len(best_params) < 5 or f1_score > min(best_params.values()):
            if len(best_params) == 5:
                # eliminar la combinación de parámetros con el peor resultado
                worst_key = min(best_params, key=best_params.get)
                del best_params[worst_key]   
            # agregar la nueva combinación de parámetros y su resultado al diccionario
            best_params[(nsta, nlta, thr_on, thr_off)] = f1_score 
            ic(best_params)

        # Actualizar la barra de progreso
        pbar.update()

    pbar.close()

    return best_params